In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-por
!pip install pytesseract
!pip install pdf2image
!pip install PyMuPDF
!pip install gspread  gspread_dataframe
!pip install unidecode
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
import fitz
import pytesseract
from pdf2image import convert_from_path
import re
import pandas as pd
import os
from PIL import Image, ImageEnhance, ImageFilter
import io
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from unidecode import unidecode
from datetime import datetime

In [ ]:
# Função para extrair texto de um PDF
def extrair_texto_pdf(caminho_pdf):
    try:
        documento = fitz.open(caminho_pdf)
        texto_completo = ""
        for pagina in documento:
            texto_pagina = pagina.get_text()
            if texto_pagina.strip():
                #print(f"Texto extraído da página {pagina.number + 1}: {texto_pagina[:100]}...")
                texto_completo += texto_pagina
            else:
                texto_ocr = extrair_texto_ocr(pagina)
                #print(f"Texto extraído por OCR da página {pagina.number + 1}: {texto_ocr[:100]}...")
                texto_completo += texto_ocr
        return texto_completo
    except Exception as e:
        print(f"Erro ao ler o arquivo {caminho_pdf}: {e}")
        return None

# Função para extrair texto usando OCR
def extrair_texto_ocr(pagina):
    try:
        pix = pagina.get_pixmap()
        img = Image.open(io.BytesIO(pix.tobytes()))

        # Melhorar a qualidade da imagem para OCR
        img = img.convert('L')
        img = img.filter(ImageFilter.SHARPEN)
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2)

        texto = pytesseract.image_to_string(img, lang='por')
        return texto
    except Exception as e:
        print(f"Erro ao realizar OCR: {e}")
        return ""

# Função para extrair a Id. do Documento e valor do texto extraído
def extrair_razao_social_valor(texto):
    padroes_razao_social = [
        #r"Descrição:\s*(.*?)\s+Pagador",
        r"Descrição:\s*(.*?)(?=\n|$)",
        r"Observação:\s*(.*?)(?=\n|$)",
        r"Beneficiário\s*([A-Z\s]+?)(?:\s{2,}|\n|$)",  # Padrão ajustado para OCR
        r"Nome Fantasia:\s*([A-Z\s]+)",  # Padrão específico para 'Nome Fantasia'
        #r"Nome:\s*([A-Z\s]+)",  # Padrão para 'Nome'
        r"CNPJ\s*:\s*\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\s*([A-Z\s]+)\s*\d{11}",  # Padrão 1
        r"Razão Social\s*:\s*([A-Z\s]+)",  # Padrão 2
        r"\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\s([A-Z\s]+)\s\d{11}",  # Padrão 3
        r"CLIENTE:\s*([A-Z\s]+)(?:\n|$)",  # Padrão 4
        #r"Beneficiário\s*Nome/Razão Social:\s*([A-Z\s]+)",  # Padrão 5 (Sicoob)
        #r"Beneficiário\s+([A-Z\s]+)\s+CNPJ",  # Padrão 6
        r"Id. do Documento\s*:\s*([a-zA-Z0-9\s]+)\s+Data do Pagamento",  # Padrão 7 (UNICRED)
        r"Razao social\s*([A-Z\s]+)\nNome fantasia",  # Padrão 8 (Santander)
    ]

    padroes_valor = [
        r"Valor\s*:\s*R?\$?\s*([\d,\,\.]+)",  # Padrão ajustado para valor
        r"Valor Total do Documento\s*\n\s*R?\$?\s*([\d,\,\.]+)",  # Padrão 1
        r"Valor:\s*R?\$?\s*([\d,\,\.]+)",  # Padrão 2
        r"Valor:\s*R?\$?\s*([\d,\,\.]+)",  # Padrão 3
        r"Valor Total\s*R?\$?\s*([\d,\,\.]+)",  # Padrão 4
        r"Pago:\s*R?\$?\s*([\d,\,\.]+)",  # Padrão 5 (Sicoob)
        r"Valor do Documento\s*\n\s*R?\$?\s*([\d,\,\.]+)",  # Padrão 6
        r"Valor Total:\s*R?\$?\s*([\d,\,\.]+)",  # Padrão 7 (UNICRED)
        r"Valor total a cobrar\s*R?\$?\s*([\d,\,\.]+)",  # Padrão 8 (Santander)
    ]

    razao_social = None
    valor = None

    for padrao in padroes_razao_social:
        match = re.search(padrao, texto, re.IGNORECASE)
        if match:
            razao_social = match.group(1).strip().replace('\n', ' ')
            # Remover ' C CPF' ou ' C Nome' se presente no texto
            razao_social = re.sub(r'\sC\s(?:CPF|Nome)$', '', razao_social)
            break

    for padrao in padroes_valor:
        match = re.search(padrao, texto, re.IGNORECASE)
        if match:
            valor = match.group(1).strip()
            break

    return razao_social, valor

# Caminho para a pasta 'documentos'
caminho_pasta = "drive/MyDrive/Códigos/BS/documentos"

# Lista para armazenar os dados extraídos
dados = []
# Lista para registrar PDFs não processados
arquivos_nao_processados = []

# Listar todos os arquivos PDF na pasta 'documentos'
for nome_arquivo in os.listdir(caminho_pasta):
    if nome_arquivo.lower().endswith('.pdf'):
        caminho_pdf = os.path.join(caminho_pasta, nome_arquivo)
        texto = extrair_texto_pdf(caminho_pdf)
        if texto:
            #print(f"Texto extraído do arquivo {nome_arquivo}:\n{texto[:500]}...")
            razao_social, valor = extrair_razao_social_valor(texto)
            if razao_social and valor:
                dados.append([razao_social, valor])
            else:
                #print(f"Falha ao extrair razão social e valor do arquivo {nome_arquivo}")
                arquivos_nao_processados.append(nome_arquivo)
        else:
            #print(f"Falha ao extrair texto do arquivo {nome_arquivo}")
            arquivos_nao_processados.append(nome_arquivo)

df = pd.DataFrame(dados, columns=["Razão Social", "Valor"])
df['Razão Social'] = df['Razão Social'].apply(unidecode)
df['Razão Social'] = df['Razão Social'].str.title()

# Substituir vírgula por ponto nos valores e converter para float
df['Valor'] = df['Valor'].str.replace('.', '')
df['Valor'] = df['Valor'].str.replace(',', '.').astype(float)

print(df)

# Exibir arquivos não processados
if arquivos_nao_processados:
    print("\nArquivos não processados:")
    for arquivo in arquivos_nao_processados:
        print(arquivo)

         Razão Social    Valor
0   Bs D C P P L Ltda   446.61
1                Ceub   599.07
2               Nvoip   280.00
3               Caesb   332.94
4               Nvoip   280.00
5      Priscila Pires  7066.27
6         Hugo Bittar  5555.81
7        Vitor Ramiro  3000.00
8             Arkmeds   898.70
9         Aluguel Kit  1726.45
10           Diasorin  5347.75
11            Novacia   379.90
12           Transrep  1129.80
13          Gelo Seco   403.13
14  Bs D C P P L Ltda   266.01
15  Bs D C P P L Ltda   127.20


In [ ]:
# Nomes para serem agrupados em 'Folha de Pagamento'
folha_pgto = ['Adriana Cesar','Adriana Santos','Andre Honorato','Andrei Felice','Danilo Cavalcante','Debora Palma','Debora Souza','Elainne Carneiro'
,'Elizangela Gonçalves','Giovani Araujo','Guilherme Borges',
              'Henrique Araujo','Hugo Bittar','Janaina Monte','Jaqueline Osiro','Joao Falcao','Kelvin Kalil','Leonardo Ferreira','Luana Coutinho'
              ,'Lucca Paz','Luiz Bonazza','Marcelo Teruo',
              'Marcus Vinicius','Matheus Lopes','Mauro Oliveira','Priscila Pires','Raquel Cardoso','Ricardo Rosa','Robson Delai','Rodrigo Alves','Vitor Ramiro',
              'Wanderley Buim','William Ribas']
# Identificar quais registros precisam ser agrupados
df['Agrupar'] = df['Razão Social'].apply(lambda x: 'Folha de Pagamento' if x in folha_pgto else 'Outro')
# Separar os dados que serão agrupados e os que não serão
df_agrupado = df[df['Agrupar'] == 'Folha de Pagamento'].groupby('Agrupar', as_index=False).agg({'Valor': 'sum'})
df_nao_agrupado = df[df['Agrupar'] == 'Outro'].drop(columns='Agrupar')
# Adicionar uma linha para 'Folha de Pagamento' no dataframe não agrupado
df_nao_agrupado = pd.concat([df_nao_agrupado, pd.DataFrame({'Razão Social': ['Folha de Pagamento'], 'Valor': df_agrupado['Valor']})], ignore_index=True)
# Selecionar apenas as colunas 'Razão Social' e 'Valor'
df_final = df_nao_agrupado[['Razão Social', 'Valor']]
# Obter a data do último mês
last_month = (datetime.now().replace(day=1) - pd.DateOffset(months=1)).strftime('%m/%Y')
df_final['Data'] = last_month
df_final['Descrição'] = df_final['Razão Social'].apply(lambda x: 'Difal' if x == 'Bs D C P P L Ltda' else x)
df_final['Razão Social'] = df_final['Razão Social'].replace({'Bs D C P P L Ltda':'Impostos'})

In [ ]:
df_final

,Razão Social,Valor,Data,Descrição
0,Impostos,446.61,06/2024,Difal
1,Ceub,599.07,06/2024,Ceub
2,Nvoip,280.00,06/2024,Nvoip
3,Caesb,332.94,06/2024,Caesb
4,Nvoip,280.00,06/2024,Nvoip
5,Arkmeds,898.70,06/2024,Arkmeds
6,Aluguel Kit,1726.45,06/2024,Aluguel Kit
7,Diasorin,5347.75,06/2024,Diasorin
8,Novacia,379.90,06/2024,Novacia
9,Transrep,1129.80,06/2024,Transrep


In [ ]:
# Caminho para o arquivo JSON das credenciais
creds_path = 'drive/MyDrive/Códigos/BS/credenciais/bs-diagnostica-723b2b3daeca.json'

# Autorizar a API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(creds_path, scopes=scope)
client = gspread.authorize(creds)

# Abrir a planilha pelo nome
sheet = client.open("teste").sheet1

# Encontrar a próxima linha vazia
def get_next_available_row(sheet):
    str_list = list(filter(None, sheet.col_values(1)))
    return len(str_list) + 1

next_row = get_next_available_row(sheet)

# Inserir o DataFrame a partir da próxima linha vazia
set_with_dataframe(sheet, df_final, row=next_row, include_column_header=False)

print("Dados adicionados com sucesso!")


Dados adicionados com sucesso!
